In [1]:

import dask
import xarray as xr
import netCDF4 as nc
import os
import csv
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import geopandas as gpd
from shapely.geometry import mapping
from pyproj import CRS
from shapely.geometry import mapping 

import rasterio as rio

from shapely.geometry import mapping
from pyproj import CRS
import rioxarray

ModuleNotFoundError: No module named 'rioxarray'

## CHAPTER 4 20CR map

In order to build my results and provide my opinion on the 20CR, I need to undertsand where and what the gridbox used for the 20th Century Reanalsys (20CR) looks like. Depending on certian factors such as how much is it over land, where is the location used for the Australian Climate Observations Reference Network – Surface Air Temperature (ACORN-SAT) compared to this grid point, is it better to use the locatin, North, South, West etc to capture the land more. This will all be discussed in this section.

But I still need to build the code up and not only that add a mask of the location onto the world map before .sel it and slice it to get the grid box I have used. The location of Perth Airport or ACORN-SAT is a latidude of -31.93deg and a longitude of 115.98deg. I have the 20CR anual mean Tmax for 1840 which I will input onto this file and show the location and the map of the Perth region in the gridbox

FYI: On my account, we MUST intall the shape-env for this map to work and save it

In [2]:
CR20_data = xr.open_mfdataset(r"E:\LIBRARY\UNIVERSITY\Masters Research\20CR\1840_Mean_Temp.nc",decode_times=True,engine='netcdf4')
#Get it to deg C
CR20_data['tmax'] = CR20_data['tmax']-273.15
CR20_data


OSError: no files to open

What we is the coordinates and the time in 366 days, so 1 value per day. Now we wish to mean this and get the annual tmax

In [4]:
Temp_Max_1840_Mean2 = CR20_data.mean(dim = ('time'))
Temp_Max_1840_Mean2

NameError: name 'CR20_data' is not defined

In [5]:
Temp_Max_1840_Mean2 = CR20_data.mean(dim = ('time'))
Temp_Max_1840_Mean2

NameError: name 'CR20_data' is not defined

In [6]:
Temp_Max_1840_Mean2['tmax'].sel(lon = slice(110,111), lat = slice(-50,-49)).plot()

NameError: name 'Temp_Max_1840_Mean2' is not defined

In [7]:
var = Temp_Max_1840_Mean2
# Shift longitudes by 0.5 degrees
#ds = var.assign_coords(lon=(((var.lon + 180) % 360) - 180) + 0.5)
# Shift the latitude values by 0.5 degrees to center the grid boxes
var['lat'] = var['lat'] + 0.5
# Shift the latitude values by 0.5 degrees to center the grid boxes
var['lon'] = var['lon'] + 0.5

#var['tmax'].sel(lon = slice(110,111), lat = slice(-50,-49)).plot()
var['tmax'].sel(lon = slice(110,111), lat = slice(-50,-49)).plot()

NameError: name 'Temp_Max_1840_Mean2' is not defined

Now what we have is a meaned version of the 1840s which makes it easier to read for this example. Remember we aim to compute the location and its grid box so this just makes it 20 times easier. So now we want to download the shape file

In [8]:
'''
This is to extract the shape file of victoria only
and using the code Mathilde sent me
using shape-env

''' 
import pandas as pd, numpy as np, xarray as xr, dask, geopandas as gpd, rasterio as rio, rioxarray, matplotlib.pyplot as plt
from shapely.geometry import mapping
from pyproj import CRS
Temp_Max_1840_Mean = Temp_Max_1840_Mean2['tmax']


crs = CRS("EPSG:4326")




world_shp = gpd.read_file(r"C:\Users\jarra\Desktop\World_Countries__Generalized_.shp")

# Mask
Temp_Max_1840_Mean.rio.set_spatial_dims(x_dim="lon", y_dim="lat", inplace=True)
Temp_Max_1840_Mean.rio.write_crs("epsg:4326",inplace=True)

# Clip
Temp_Max_1840_Mean = Temp_Max_1840_Mean.rio.clip(world_shp.geometry.apply(mapping))

#Check if there is a working map
Temp_Max_1840_Mean.sel(lon = slice(112,122), lat = slice(-34,-29)).plot()
Temp_Max_1840_Mean.encoding['grid_mapping'] = 'latitude-longitude'
#crs:grid_mapping_name = "latitude_longitude"



ModuleNotFoundError: No module named 'rioxarray'

In [9]:
import os,sys
from IPython.display import Image
jpgfile = Image(r"C:\Users\jarra\Desktop\rough 1 by 1 box for perth.png")
jpgfile


FileNotFoundError: No such file or directory: 'C:\Users\jarra\Desktop\rough 1 by 1 box for perth.png'

FileNotFoundError: No such file or directory: 'C:\Users\jarra\Desktop\rough 1 by 1 box for perth.png'

<IPython.core.display.Image object>

In [10]:
Temp_Max_1840_Mean.sel(lon = slice(115,117), lat = slice(-33,-31)).plot()
Temp_Max_1840_Mean.encoding['grid_mapping'] = 'latitude-longitude'

NameError: name 'Temp_Max_1840_Mean' is not defined

This is the rough image of the location of where the grix box is estimating. What we can see is that a majority of grid, more then 50% is actually in the water. Now thec coords of -31.93deg and a longitude of 115.98deg say that the Airport is in the bottom left of the photo which will actually affect how heatwaves are identified as in the grid box above where its showing the mid point temperature at the edges of the grid cell. Even with half a degree shift, at least 30 to 40% of the temperature average will still be from the ocean therefore affecting the strength of temperature changes compared to land. Why is this the case, well we have to go back to basic chemistry and undertsand how specific heat capacitiy works. Different substances have different heat capacitiies and the higher the value the longer it takes to warm up as more enegery is required to heat it up by 1 degree and in turn the longer it takes to cool compared to something with a small heat capacity. The land heats up quickly and cools quicker. This essentially causes the fremantle doctor sea breeze if the difference in temperature between the land and the ocean is significant.

Now what we know is that the tnmax in the above grid cell shows cooler then average temperatures where the land meets ocean generally has significantly cooler temperatures then inland temperature and the white areas are ocean regions. This shows that choosing a gridbox will be affected by its entire regional temperatures that have been inputted into the system to generate the 20CR tmax therefore for this specific grid box it will be affected by the slower temperature changes in the ocean then the land causing cooler then nromal temperatures for coastal gridboxes.

So what best represents the ACORN-SAT, well we know that the gridbox would have been the best estimate, however of coord -31.93deg and a longitude of 115.98deg actually lands the temperature calcuation in the middle of the grid box estimate therefore there isnt really a lot of choice we can do going forward. The bnds are set from 0 to 1 degree and temperature for that gridbox is set at the middle so 0.5 with the 1 deg box that encapslulates this value so the temperature projected is at 0.5 to 1.5 due to the temperature being recorded at the outer egde of that box ??? but overall, I dont think there is any reason to change it to the east by 1 degree as it will definitely throw the temperatures out of proportion with the observational data. So we will stick with the defined grid box that I have used for my heatwave analysis that will be explained in detail later on (SECTION)

In [11]:
#TESTER
#%%
import numpy as np
import xarray as xr

# Create lat and lon arrays
lats = np.arange(-89.5, 90.5, 1.0)
lons = np.arange(0.5, 360.5, 1.0)

# Create grid arrays with the mid-point of each cell
lat_bnds = np.arange(-90, 90, 1.0)
lon_bnds = np.arange(-0, 360, 1.0)

# Create 2D arrays of latitudes and longitudes
lat_2d, lon_2d = np.meshgrid(lat_bnds, lon_bnds, indexing='ij')

# Create data array with random values between 0 and 50
tmax = xr.DataArray(np.random.uniform(low=0.0, high=50.0, size=(len(lats), len(lons))),
                    coords=[lats, lons], dims=["lat", "lon"])

# Create dataset with lat, lon, and tmax
THETEST = xr.Dataset({"tmax": tmax}, coords={"lat_bnds": (("lat", "lon"), lat_2d),
                                         "lon_bnds": (("lat", "lon"), lon_2d),
                                         "lat": lats,
                                         "lon": lons})
THETEST


<xarray.Dataset>
Dimensions:   (lat: 180, lon: 360)
Coordinates:
  * lat       (lat) float64 -89.5 -88.5 -87.5 -86.5 ... 86.5 87.5 88.5 89.5
  * lon       (lon) float64 0.5 1.5 2.5 3.5 4.5 ... 356.5 357.5 358.5 359.5
    lat_bnds  (lat, lon) float64 -90.0 -90.0 -90.0 -90.0 ... 89.0 89.0 89.0 89.0
    lon_bnds  (lat, lon) float64 0.0 1.0 2.0 3.0 4.0 ... 356.0 357.0 358.0 359.0
Data variables:
    tmax      (lat, lon) float64 17.27 3.224 47.84 11.36 ... 38.01 23.55 10.36

In [12]:
Temp_Max_1840_Mean2


NameError: name 'Temp_Max_1840_Mean2' is not defined